In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))
import lightgbm as lgb

df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")


# Any results you write to the current directory are saved as output.

In [ ]:
# # 相関係数の確認
# df = df_train
# corr_mat = df.corr(method='pearson')
# corr_mat

df_train["Title"] = df_train.Name.str.extract("([A-Za-z]+)\.", expand=False)
df_test["Title"] = df_test.Name.str.extract("([A-Za-z]+)\.", expand=False)

#Titleから新しい特徴量を追加
def title_to_num(title):
    if title == "Master":
        return 1
    elif title == "Miss":
        return 2
    elif title == "Mr":
        return 3
    elif title == "Mrs":
        return 4
    else:
        return 5
    
df_train["Title_num"]=[title_to_num(i) for i in df_train["Title"]]
df_test["Title_num"]=[title_to_num(i) for i in df_test["Title"]]

df_train.head()

# corr_mat = df_train.corr(method='pearson')
# corr_mat


In [ ]:
#年齢の補完(nameTitleを使ってやるとなお良いと思う)
#Ageがnullの場合に、　Pclassに応じてAgeに代入する関数

#PclassごとにAgeの平均を算出
print(df_train.groupby("Title_num").mean()["Age"])

def impute_age(cols):
    Age = cols[0]
    Title_num = cols[1]
    Pclass = cols[2]
    
    if pd.isnull(Age):
        if Title_num == 1:
            return 4.6
        elif Title_num == 2:
            return 21.8
        elif Title_num == 3:
            return 32.4
        elif Title_num == 4:
            return 35.9
        elif Title_num == 5:
            if Pclass == 1:
                return 39
            elif Pclass == 2:
                return 30
            else:
                return 25
    else:
        return Age
    
df_train["Age"] = df_train[["Age", "Title_num", "Pclass"]].apply(impute_age, axis = 1)
df_test["Age"] = df_test[["Age", "Title_num", "Pclass"]].apply(impute_age, axis = 1)


In [ ]:
#乗港した港の補完
df_train[df_train["Embarked"].isnull()]
# 同じticket番号の人他にいないか確認
df_test[df_test["Embarked"]=="113572"]
#一番多いSで埋める
df_train.loc[df_train["PassengerId"].isin([62,830]),"Embarked"]="S"
df_train.loc[df_train["PassengerId"].isin([62,830])]

In [ ]:
# 運賃の補完
df_train[["Pclass","Fare"]].groupby("Pclass").mean()

df_test[df_test["Fare"].isnull()]

df_test.loc[df_test["PassengerId"]==1044,"Fare"]=13.675550
df_test.loc[df_test["PassengerId"]==1044]

In [ ]:
#性別のダミー変数化
df_train = pd.get_dummies(df_train, columns=["Sex"])
df_test = pd.get_dummies(df_test, columns=["Sex"])

# Embarkedはダミー変数ではなくて、数値に置き換え
# df_train = pd.get_dummies(df_train, columns=["Embarked"])
# df_test = pd.get_dummies(df_test, columns=["Embarked"])
df_train["Embarked"] = df_train["Embarked"].map({"S":0,"C":1,"Q":2}).astype(int)
df_test["Embarked"] = df_test["Embarked"].map({"S":0,"C":1,"Q":2}).astype(int)

df_train.head()

In [ ]:
# # AgeとFareをカテゴリ変数化
data = [df_train, df_test]
for df in data:
    #Ageのカテゴリ変数化
    df.loc[df["Age"] <= 16, "Age"] = 0
    df.loc[(df["Age"] > 16) & (df["Age"] <= 32), "Age"] = 1
    df.loc[(df["Age"] > 32) & (df["Age"] <= 48), "Age"] = 2
    df.loc[df["Age"] > 48, "Age"] = 3
    df["Age"] = df["Age"].astype("int")
    #Fareのカテゴリ変数化
    df.loc[df["Fare"] <= 7.91, "Fare"] = 0
    df.loc[(df["Fare"] > 7.91) & (df["Fare"] <= 14.454), "Fare"] = 1
    df.loc[(df["Fare"] > 14.454) & (df["Fare"] <= 31), "Fare"] = 2
    df.loc[df["Fare"] > 31, "Fare"] = 3
    df["Fare"] = df["Fare"].astype("int")
    
    #FamilySizeとIsAloneの作成
    df["FamilySize"] = df["SibSp"] + df["Parch"] +1
    df["IsAlone"] = 0
    df.loc[df["FamilySize"] == 1, "IsAlone"] = 1
    
df_train.head()

In [ ]:
df_train.drop(["Name","Cabin","Ticket","SibSp","Parch", "Title"], axis=1, inplace=True)
df_test.drop(["Name","Cabin","Ticket","SibSp","Parch", "Title"], axis=1, inplace=True)

df_train.head()

In [ ]:
X_train = df_train.drop(["PassengerId", "Survived"], axis=1)
Y_train = df_train["Survived"]
X_test = df_test.drop("PassengerId", axis=1).copy()

In [ ]:
# #グリッドサーチの範囲を指定
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestClassifier
# import sklearn 

# parameters = {
#     "n_estimators":[i for i in range(10,100,10)],
#     "criterion":["gini","entropy"],
#     "max_depth":[i for i in range(1,6,1)],
#      'min_samples_split': [2, 4, 10,12,16],
#     "random_state":[3],
# }
# #モデルを作成
# clf = sklearn.model_selection.GridSearchCV(RandomForestClassifier(), parameters,cv=5,n_jobs=-1)
# clf_fit=clf.fit(X_train, Y_train)
# #最も良い学習モデルで学習
# predictor=clf_fit.best_estimator_



In [ ]:
# k分割交差検証を用いて学習
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=3)

scores = []

# for train_index, test_index in kf.split(X_train, Y_train):
#     X_cv_train = X_train.iloc[train_index]
#     X_cv_test = X_train.iloc[test_index]
#     Y_cv_train = Y_train.iloc[train_index]
#     Y_cv_test = Y_train.iloc[test_index]
#     forest = RandomForestClassifier(random_state=1)
#     forest.fit(X_cv_train, Y_cv_train)
#     predictions = forest.predict(X_cv_test)
#     score = round(accuracy_score(Y_cv_test, forest.predict(X_cv_test))*100, 2)
#     scores.append(score)
#     print(score)

for train_index, test_index in kf.split(X_train, Y_train):
    X_cv_train = X_train.iloc[train_index]
    X_cv_test = X_train.iloc[test_index]
    Y_cv_train = Y_train.iloc[train_index]
    Y_cv_test = Y_train.iloc[test_index]
    
    #識別器を作成
    gbm = lgb.LGBMClassifier(objective="binary",
                             learning_rate=0.1, 
                             max_depth=2, 
                             n_estimators=1500, 
                             num_leaves=300
                            )
    #学習
    gbm.fit(X_cv_train, Y_cv_train,
           eval_set = [(X_cv_test, Y_cv_test)],
           early_stopping_rounds=10,)
    
    #予測し制度を表示
    Y_pred = gbm.predict(X_cv_test, num_iteration=gbm.best_iteration_)
    score = round(accuracy_score(Y_cv_test, Y_pred)*100, 2)
    scores.append(score)
    
print(sum(scores) / 3)

In [ ]:
grid_param ={'n_estimators':[1500,1000],
             'max_depth':[3,2],
             'num_leaves':[300,350],
             'learning_rate':[0.1,0.15]
            }

fit_params={'early_stopping_rounds':10, 
            'eval_metric' : 'rmse', 
            'eval_set' : [(X_train, Y_train)]
           }
bst_gs_cv = GridSearchCV(
            lgb.LGBMClassifier(), # 識別器
            grid_param, # 最適化したいパラメータセット 
            cv = KFold(n_splits=3, shuffle=True), # 交差検定の回数
            scoring = 'neg_mean_squared_error',
            verbose = 0
            )

bst_gs_cv.fit(X_train, Y_train,**fit_params,verbose = 0)

best_param = bst_gs_cv.best_params_
print('Best parameter: {}'.format(best_param))

In [ ]:
# 特徴量の重要度を確認
for i,k in zip(X_train.columns,gbm.feature_importances_):
    print(i,round(k,4))

In [ ]:
#提出
Y_prediction = gbm.predict(X_test)
submission = pd.DataFrame({
    "PassengerId": df_test["PassengerId"],
    "Survived": Y_prediction
    })
submission.to_csv("lgb_submission.csv", index=False)